In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import * 

In [0]:
%run ../utility/delta_merge

In [0]:
silver_data_source  = spark.read.table('lakehouse.`03_silver`.earthquake_events')


In [0]:
silver_data_source.display()

In [0]:
#TODO
# 1. event_density per location


In [0]:
event_density = (
    silver_data_source
    .groupBy(
        F.round(F.col('longitude'), 1).alias('avg_long'),
        F.round(F.col('latitude'), 1).alias('avg_lat'),
    )
    .agg(
        F.count("*").alias("event_density"),
        F.avg("mag").alias("avg_mag"),
        F.max("mag").alias("max_mag")
    )
    .withColumn('hash_id', F.sha2(F.concat_ws('_', F.col('avg_long'), F.col('avg_lat')), 256).substr(0,15))
)

event_density.display()

In [0]:
table_name = 'lakehouse.`04_gold`.event_density'

In [0]:
if check_delta_table(table_name):
    print(f"{table_name} table exists in the catalog, Updating it")
    delta_merge(event_density, table_name, "hash_id")
else:
    print(f"{table_name} table does not exist in the catalog, Writing it for the first time")
    event_density.write.mode("overwrite").format("delta").option("delta.enableChangeDataFeed", "true").saveAsTable(table_name)

In [0]:
%sql
select * from lakehouse.`04_gold`.event_density
-- group by hash_id;